In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_Correction.csv')

In [26]:
df.head()

,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,Verb Tense Errors,I goes to the store everyday.,I go to the store everyday.
1,2,Verb Tense Errors,They was playing soccer last night.,They were playing soccer last night.
2,3,Verb Tense Errors,She have completed her homework.,She has completed her homework.
3,4,Verb Tense Errors,He don't know the answer.,He doesn't know the answer.
4,5,Verb Tense Errors,The sun rise in the east.,The sun rises in the east.


In [27]:
# Checking for nulls
df.isnull().sum()

,0
Serial Number,0
Error Type,0
Ungrammatical Statement,0
Standard English,0


In [28]:
# Converting all uppercase to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-28-d4a5b9817d45>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [29]:
# Function to retain only specific punctuation
#  Retains only word characters, whitespace, and . , ? !
def retain_selected_punctuation(text):
    if isinstance(text, str):
        cleaned_text = re.sub(r"[^\w\s.,?!']", '', text)
        return cleaned_text
    else:
        return text

# Apply the function to the 2 columns
df['Ungrammatical Statement'] = df['Ungrammatical Statement'].apply(retain_selected_punctuation)
df['Standard English'] = df['Standard English'].apply(retain_selected_punctuation)

df.head()

,Serial Number,Error Type,Ungrammatical Statement,Standard English
0,1,verb tense errors,i goes to the store everyday.,i go to the store everyday.
1,2,verb tense errors,they was playing soccer last night.,they were playing soccer last night.
2,3,verb tense errors,she have completed her homework.,she has completed her homework.
3,4,verb tense errors,he don't know the answer.,he doesn't know the answer.
4,5,verb tense errors,the sun rise in the east.,the sun rises in the east.


In [30]:
# Tokenization
def tokenize_text(text):
    """
    Tokenizes text into words (for sentence-level analysis).
    """
    if isinstance(text, str):
        # Tokenize the text into words
        word_tokens = nltk.word_tokenize(text)
        return word_tokens
    else:
        return []

# Apply tokenization to both columns
df['Tokenized Ungrammatical'] = df['Ungrammatical Statement'].apply(tokenize_text)
df['Tokenized Standard'] = df['Standard English'].apply(tokenize_text)

# Display the resulting DataFrame
df.head()


,Serial Number,Error Type,Ungrammatical Statement,Standard English,Tokenized Ungrammatical,Tokenized Standard
0,1,verb tense errors,i goes to the store everyday.,i go to the store everyday.,"[i, goes, to, the, store, everyday, .]","[i, go, to, the, store, everyday, .]"
1,2,verb tense errors,they was playing soccer last night.,they were playing soccer last night.,"[they, was, playing, soccer, last, night, .]","[they, were, playing, soccer, last, night, .]"
2,3,verb tense errors,she have completed her homework.,she has completed her homework.,"[she, have, completed, her, homework, .]","[she, has, completed, her, homework, .]"
3,4,verb tense errors,he don't know the answer.,he doesn't know the answer.,"[he, do, n't, know, the, answer, .]","[he, does, n't, know, the, answer, .]"
4,5,verb tense errors,the sun rise in the east.,the sun rises in the east.,"[the, sun, rise, in, the, east, .]","[the, sun, rises, in, the, east, .]"
